In [ ]:
from keras.layers import Input, Dense, Reshape, Flatten
from keras.layers import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
#from keras.optimizers import Adam
import tensorflow as tf
#kb.set_session(kb.tf.Session(config=kb.tf.ConfigProto(intra_op_parallelism_threads=12, inter_op_parallelism_threads=12)))
#tf.config.threading.set_intra_op_parallelism_threads(12)
#tf.config.threading.set_inter_op_parallelism_threads(12)

import mdtraj as md
import numpy as np
import keras as krs


In [ ]:
refpdb = md.load_pdb("md_ala_final_noPBC_fit_noH.pdb")
X = md.load("md_ala_final_noPBC_fit_noH.xtc", top="md_ala_final_noPBC_fit_noH.pdb")
X.superpose(refpdb)
Xt = np.zeros((X.n_frames, X.n_atoms*3, 1))
for i in range(X.n_frames):
  for j in range(X.n_atoms):
    Xt[i,3*j,0] = X.xyz[i,j,0]
    Xt[i,3*j+1,0] = X.xyz[i,j,1]
    Xt[i,3*j+2,0] = X.xyz[i,j,2]
X_train = Xt
scale = [np.min(X_train), np.max(X_train)]
X_train = (X_train - scale[0])/(scale[1] - scale[0])



In [ ]:
input_coord = krs.layers.Input(shape=(3*X.n_atoms,))
encoded = krs.layers.Dense(32, activation='sigmoid', use_bias=True)(input_coord)
encoded = krs.layers.Dense(32, activation='sigmoid', use_bias=True)(encoded)
encoded = krs.layers.Dense(2, activation='linear', use_bias=True)(encoded)

decoded = krs.layers.Dense(32, activation='sigmoid', use_bias=True)(encoded)
decoded = krs.layers.Dense(32, activation='sigmoid', use_bias=True)(decoded)
decoded = krs.layers.Dense(3*X.n_atoms, activation='linear', use_bias=True)(decoded)
codecvs = krs.models.Model(input_coord, decoded)
encoder = krs.models.Model(input_coord, encoded)
codecvs.compile(optimizer='Adam', loss='mean_squared_error')

In [ ]:
history = codecvs.fit(X_train, X_train,
            epochs=1000,
            batch_size=256,
            validation_data=(X_train, X_train))

In [ ]:
coded_ala = encoder.predict(X_train)
np.savetxt("out.txt", coded_ala)
np.savetxt("loss.txt", history.history['loss'])



In [ ]:
decoded_ala = codecvs.predict(X_train)
decoded_ala = decoded_ala*(scale[1] - scale[0]) + scale[0]
np.savetxt("outc.txt", decoded_ala)